In [1]:
from pennylane import numpy as np
import pennylane as qml

In [5]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

def multisol_oracle_matrix(combos):
    """Return the oracle matrix for a set of solutions.

    Args:
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    indices = [np.ravel_multi_index(combo, [2]*len(combo)) for combo in combos]
    ##################
    # YOUR CODE HERE #
    ##################
    my_array = np.identity(2**len(combos[0])) # Create the identity matrix
    for index in indices:
        my_array[index, index] = -1
    return my_array

@qml.qnode(dev)
def multisol_pair_circuit(x_tilde, combos):
    """Implements the circuit for testing a pair of combinations labelled by x_tilde.
    
    Args:
        x_tilde (list[int]): An (n_bits - 1)-bit string labelling the pair to test.
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    ##################
    # YOUR CODE HERE #
    ##################
    qml.Hadamard(wires=n_bits-1)
    qml.QubitUnitary(oracle_matrix(combos), wires=range(n_bits))
    qml.Hadamard(wires=n_bits-1)

    return qml.probs(wires=n_bits-1)

In [6]:
def parity_checker(combos):
    """Use multisol_pair_circuit to determine the parity of a solution set.

    Args:
        combos (list[list[int]]): A list of secret combinations.

    Returns: 
        int: The parity of the solution set.
    """
    parity = 0
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]
    for x_tilde in x_tildes:

        ##################
        # YOUR CODE HERE #
        ##################

        # IMPLEMENT PARITY COUNTING ALGORITHM
        prob_of_one = multisol_pair_circuit(x_tilde, combos)[1]
        if np.isclose(prob_of_one, 1.0):
            parity += 1
                
    return parity % 2


In [4]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """Test a pair labelled by x_tilde for the presence of a solution.
    
    Args:
        x_tilde (list[int]): An (n_bits - 1)-string labelling the pair to test.
        combo (list[int]): A secret combination of n_bits 0s and 1s.
        
    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    ##################
    # YOUR CODE HERE #
    ##################
    qml.Hadamard(wires=n_bits-1)
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.Hadamard(wires=n_bits-1)
    return qml.probs(wires=n_bits-1)


In [21]:
def secret_combo(n_bits):
    x_tilde_strs = [np.binary_repr(n, n_bits) for n in range(2**(n_bits))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]
    #print(x_tildes)
    rand_idx = np.random.choice(range(len(x_tildes)), size=1)[0]
    #print(rand_idx)
    return x_tildes[rand_idx]


In [24]:
[secret_combo(n_bits) for i in range(10)]

[[1, 0, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 0],
 [1, 1, 1, 0],
 [1, 0, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 1, 0],
 [0, 1, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 1, 0]]

In [26]:
combo = secret_combo(n_bits)

In [28]:
combo

[0, 0, 0, 0]

In [32]:
x_tilde = [0,0,0]

In [35]:
pair_circuit(x_tilde, combo)[1]

tensor(1., requires_grad=True)

In [37]:
def pair_lock_picker(trials):
    """Create a combo, run pair_circuit until it succeeds, and tally success rate.
    
    Args:
        trials (int): Number of times to test the lock picker.

    Returns:
        float: The average number of times the lock picker uses pair_circuit.
    """
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs] 

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits) # Random list of bits
        counter = 0
        for x_tilde in x_tildes:
            counter += 1

            ##################
            # YOUR CODE HERE #
            ##################
            prob_of_one = pair_circuit(x_tilde, combo)[1]
            if np.isclose(prob_of_one, 1.0):
                break
        test_numbers.append(counter)
    return sum(test_numbers)/trials

In [38]:
trials = 1000
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")

For 4 bits, it takes 4.497 pair tests on average.
